In [2]:
def parent_func():
    def child_func(x =100):
        print(x * 1.08)
    return child_func

In [3]:
output = parent_func()

In [4]:
output()

108.0


In [5]:
output(23)

24.840000000000003


In [7]:
print(output())

108.0
None


In [27]:
def counter(func):
  """This is a docstring for parent_func, counter"""
  def wrapper(*args, **kwargs):
    """This is a docstring for child_func, wrapper"""
    wrapper.count += 1
    # Call the function being decorated and return the result
    return func(*args, **kwargs)
  # Set count to 0 to initialize call count for each new decorated function
  wrapper.count = 0
  # Return the new decorated function
  return wrapper

In [ ]:

# Decorate foo() with the counter() decorator
@counter
def foo():
  print('calling foo()')
  
foo()
foo()

print('foo() was called {} times.'.format(foo.count))

In [28]:
def foo():
  """This is a docstring for simple foo, outside"""
  print('calling foo()')

In [29]:
foo = counter(foo)

In [11]:
foo.__closure__

(<cell at 0x1102eb400: function object at 0x1102e65c0>,
 <cell at 0x1102e8280: function object at 0x1102e6340>)

In [30]:
doo = foo.__closure__[0].cell_contents

In [31]:
print(doo.__code__.co_varnames)        # Local variable names
print(doo.__code__.co_argcount)        # Number of positional arguments
print(doo.__code__.co_consts)        # Constants in the function
print(doo.__code__.co_names)        # All names used (functions, vars, etc.)
print(doo.__code__.co_filename)        # Source file name
print(doo.__code__.co_firstlineno)     # Line number where the function starts


()
0
('This is a docstring for simple foo, outside', 'calling foo()', None)
('print',)
/var/folders/n0/9wldyr854817hc82h7w85z6c0000gn/T/ipykernel_9012/1333178602.py
1


In [32]:
doo.__doc__

'This is a docstring for simple foo, outside'

In [ ]:
#I could not understand memoize wraper. I am not understanding counter wrapper. Is there someting wrong with my brain? What the hell am I stuck. Let me try to understand this.

def counter(func):
    """This is a docstring for parent_func, counter"""
    def wrapper(*args, **kwargs):
        """This is a docstring for child_func, wrapper"""
        wrapper.count += 1
        # What is wrapper.count? How is a function able to have a count and how is it being defined within t
        # Call the function being decorated and return the result
        return func(*args, **kwargs)
    
    wrapper.count = 0
    # The wrapper count 
    # Return the new decorated function
    return wrapper

In [ ]:
def memoize(func):
    """
    Store the results of the decorated function for fast lookup
    """
    # Store results in a dict that maps arguments to results
    cache = {}

    # Define the wrapper function to return.
    def wrapper(*args, **kwargs):
        
        kwargs_key = tuple(sorted(kwargs.items()))
        print("Now printing kwargs_key: ", kwargs_key)

        if (args, kwargs_key) not in cache:
            print(f"These values {(args, kwargs_key)} are not there in the cache", )
            cache[(args, kwargs_key)] = func(*args, **kwargs)
        return cache[(args, kwargs_key)]

    return wrapper


In [2]:
import time
def dummy_function(a, b):
    x = a + b
    y = a*b
    print(x*y)
    time.sleep(5)
    print("Aaj ki neend samapt hui")
    print(y/x)

In [ ]:
tryfun1 = memoize(dummy_function)

# The function called wrapper just returns the value of the function result call. For ex -

# I got into a tailspin. Here the main function can be either a returning function or a non returning function.
# By returning function I mean, it will return some value. The dummy_function I defined however will not ahve a return value. It just prints 3 things in quick succession. Let me first analyse how the memoize ecorator behaves in case i give it a function with no return value. then I will get to function with return value.

In [5]:
tryfun1.__closure__[0].cell_contents

{}

In [6]:
dummy_function(2,3)

30
Aaj ki neend samapt hui
1.2


In [8]:
# When I call dummy_function normally, it will always take 5 seconds to execute. (even if inputs was the same)

In [7]:
dummy_function(2,3)

30
Aaj ki neend samapt hui
1.2


In [10]:
# When I do, tryfun1 = memoize(dummy_function), I am creating a new function called tryfun1, using the doubly nested memoize function.
# This tryfun1 is a completely different function that got created when I called 
# tryfun1 = memoize(dummy_function)
# Now what does memoize(dummy_function) do, it takes in dummy_function as argument. it knows that dummy function has 2 args called a and b.
# Then a nonlocal variable called cache is created. This variable is an empty dictionary.
# Then something happens ( a beautiful thing)
# Once the beautiful thing is complete, memoize returns a thing.
# The thing that memoize returns is the beautiful thing

In [11]:
# The beautiful thing is the definition of a new function called wrapper.
# This new function doesn't have much within it. 
# It is meant to replicate the behaviour of the function that was brought in as the input to the memoize function.
# It will take the same arguments as the input function. This is done like "def wrapper(*args, **kwargs)"
# This thing somehow knows that the arguments to be taken are from the input function.

In [ ]:
# The cache is stored in the memory location of tryfunc1. tryfunc1 has taken on the form of the wrapper fucntion and hence has its closure. So whenver I call tryfun1, it can access the cache variable. It is stored in memory run time within scope of tryfunc1. It can't be accessed outside of tryfunc1.

What variables from the outer scope are used inside wrapper?

    func: used to call the original function

    cache: used to store results

Therefore:

    Both func and cache are free variables inside wrapper.

    So both will be stored in wrapper.__closure__

In [12]:
@memoize
def slow_function(a, b):
    return a + b

for i, cell in enumerate(slow_function.__closure__):
    print(f"Closure cell {i}: {cell.cell_contents}")


Closure cell 0: {}
Closure cell 1: <function slow_function at 0x108877420>


In [13]:
print(slow_function.__code__.co_freevars)
# Output: ('func', 'cache')


('cache', 'func')


Yes, __closure__[i].cell_contents can contain the original function, if it's referenced inside the inner function.

In your case, it does — because func is used inside wrapper.